# Part 1: analyze demos

In [1]:
import os, sys
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW

db_name = 'rtcw_db.db'
demos_folder = 'demos'

cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
demos_folder = 'demos'
root_path = 'C:\\Users\\JelleGrammens\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [4]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
player_df = db.get_table_as_df(table_name= 'player')
obituary_df = RTCW.add_match_data(obituary_df, player_df, demos_dct)

obituary table fetched in 0.38 seconds
player table fetched in 0.03 seconds


In [5]:
obituary_df = obituary_df.loc[obituary_df['League'] == 'bob1']

In [25]:
#example, get all 5 man panzerfausts
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                maxtime_secs = 2, 
                                weapon_filter = ['grenade'], 
                                minspree = 3,
                                verbose = True)

all done!


In [53]:
df_spree

,attacker,demo,end,md5,player,spreecount,start
0,8,BOB_Empire_vs_Krieg_02_28_2002.dm_60,42084600,204e36f690b74c4d0c2d9ed4d6e10756,III-vort3x,3,42084600
1,3,Avenger.dm_60,48416600,1043519499f3d08ef9e10cfd11e9f3fe,-wm-IdiotSavant-,3,48416600
2,14,dsvsdrmatch.dm_60,14323450,edb8efea82067ecff1b07380b0fe1de2,D|S-sh4ms,3,14323450
3,4,1012528474Flipside79UVWvs86.dm_60,37655650,70878f3907cae6cfcc70c5d8cdd8c398,[Ars]Kaeto,3,37655650
4,9,BOB_Empire_vs_Doctors_03_07_2002.dm_60,42025300,92a5e15597eb8276b372f1276c757197,III-Fredo,3,42025300


In [62]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 10, offset_end = 10)

In [ ]:
for demo in df_spree.md5:
    for k in demos_dct:
        if demo in demos_dct[k][0]:
            print k

# Part 2: cut demos

In [ ]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 5, offset_end = 5)

In [ ]:
def locate_demo_path(demos_dct, spree, root_path):
    '''
    helper function that outputs the path to the demo that we want to cut
    '''
    for k in demos_dct:
        if spree.md5 in demos_dct[k][0]:
            match_folder = k
            demo_loc = [i for i in range(len(demos_dct[k][0])) if demos_dct[k][0][i]==spree.md5]
            demo_name = demos_dct[k][1][demo_loc[0]]
            
            return match_folder, demo_name

In [ ]:
def cutter_exe_cmd(root_path, match_folder, demo_name, start_time, end_time, 
                   demo_folder_name = 'demos', output_folder = 'output_spree_demos', cut_type = 1):
    '''
    helper function to create string with demo_path and parameters to input in anders libtech 3 api
    parameters: 
    - demo_path: full path to demo
    - parameters_dct: a dictionary with all the parameters necessary
    '''
    demo_path = os.path.join(root_path, demo_folder_name, match_folder, demo_name)
    output_path = os.path.join(root_path, output_folder, demo_name)
    
    s = 'cut ' + demo_path + ' '
    s += output_path + ' '
    s += str(start_time) + ' '
    s += str(end_time) + ' '
    s += str(cut_type) + ' 0' # plus the zero at the end or the api will crash

    return s

In [ ]:
def cut_demos(root_path, demos_dct, df_spree, exe_name, offset_start = 5000, offset_end = 5000):

    for row in range(len(df_spree)):

        spree = df_spree.loc[row]
        match_folder, demo_name = locate_demo_path(demos_dct, spree, root_path)

        start_time = spree.start - offset_start
        end_time = spree.end + offset_end

        parameters = cutter_exe_cmd(root_path, match_folder, demo_name, start_time, end_time, 
                                    demo_folder_name = 'demos', output_folder = 'output_spree_demos', cut_type = 1)

        os.system(exe_path + ' ' + parameters)

# Part 3 Create capture list for DemoViewer

In [ ]:
def temp_to_60(x):
    x = x[:-2] + '60'
    return x

df_spree['demo'] = df_spree['demo'].apply(lambda x: temp_to_60(x))

In [46]:
df_spree

,attacker,demo,end,md5,player,spreecount,start
0,8,BOB_Empire_vs_Krieg_02_28_2002.dm_60,42084600,204e36f690b74c4d0c2d9ed4d6e10756,III-vort3x,3,42084600
1,3,Avenger.dm_60,48416600,1043519499f3d08ef9e10cfd11e9f3fe,-wm-IdiotSavant-,3,48416600
2,14,dsvsdrmatch.dm_60,14323450,edb8efea82067ecff1b07380b0fe1de2,D|S-sh4ms,3,14323450
3,4,1012528474Flipside79UVWvs86.dm_60,37655650,70878f3907cae6cfcc70c5d8cdd8c398,[Ars]Kaeto,3,37655650
4,9,BOB_Empire_vs_Doctors_03_07_2002.dm_60,42025300,92a5e15597eb8276b372f1276c757197,III-Fredo,3,42025300


In [67]:
from lxml import etree

captureList = etree.Element('CaptureList')

for row in range(len(df_spree)):
    spree = df_spree.loc[row]
    capture = etree.Element('capture')
    id = etree.Element('id')
    demoPath = etree.Element('demoPath')
    startFrame = etree.Element('startFrame')
    stopFrame = etree.Element('stopFrame')
    localOffset = etree.Element('localOffset')
    selected = etree.Element('selected')
    playerPov = etree.Element('playerPov')
    config = etree.Element('config')

    id.text = '-1'
    selected.text = 'Yes'
    localOffset.text = '0'
    demoPath.text = spree.demo
    startFrame.text = str(spree.start - 50000)
    stopFrame.text = str(spree.end + 50000)
    playerPov.text = str(spree.attacker)
    config.text = 'follow' + str(spree.attacker) + '.cfg'

    capture.append(id)
    capture.append(demoPath)
    capture.append(startFrame)
    capture.append(stopFrame)
    capture.append(localOffset)
    capture.append(selected)
    capture.append(playerPov)
    capture.append(config)

    captureList.append(capture)

# pretty string
s = etree.tostring(captureList, pretty_print=True)
print s

<CaptureList>
  <capture>
    <id>-1</id>
    <demoPath>BOB_Empire_vs_Krieg_02_28_2002.dm_60</demoPath>
    <startFrame>42034600</startFrame>
    <stopFrame>42134600</stopFrame>
    <localOffset>0</localOffset>
    <selected>Yes</selected>
    <playerPov>8</playerPov>
    <config>follow8.cfg</config>
  </capture>
  <capture>
    <id>-1</id>
    <demoPath>Avenger.dm_60</demoPath>
    <startFrame>48366600</startFrame>
    <stopFrame>48466600</stopFrame>
    <localOffset>0</localOffset>
    <selected>Yes</selected>
    <playerPov>3</playerPov>
    <config>follow3.cfg</config>
  </capture>
  <capture>
    <id>-1</id>
    <demoPath>dsvsdrmatch.dm_60</demoPath>
    <startFrame>14273450</startFrame>
    <stopFrame>14373450</stopFrame>
    <localOffset>0</localOffset>
    <selected>Yes</selected>
    <playerPov>14</playerPov>
    <config>follow14.cfg</config>
  </capture>
  <capture>
    <id>-1</id>
    <demoPath>1012528474Flipside79UVWvs86.dm_60</demoPath>
    <startFrame>37605650</startFrame

In [68]:
tree = etree.ElementTree(captureList)
tree.write('output.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")